In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind

from backtrader.feeds import GenericCSVData

In [2]:
class dix_csv(GenericCSVData):
    
    lines= ('dix', 'gex')
    
    params = (
        ('dtformat', '%m/%d/%Y'),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 6),
        ('openinterest', -1),
        ('dix', 9),
        ('gex', 10),        
    )

In [5]:
#Optimize DIX MA
class DIXmaopt(bt.Strategy):
    params = (
        ('period', 2),
        ('printlog', False),
        ('dixupper', 45),
        ('dixlower', 40),
    )
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #Add a DIXSMA Indicator
        self.dixsma = bt.indicators.MovingAverageSimple(
           self.data.dix, period=self.params.period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # buy when dix greater than dixupper threshold
            if self.dixsma >= (self.params.dixupper/100):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        else:
            
            # sell after 60 trading days
            self.duration = len(self) - self.bar_executed + 1
            if self.duration >= 60:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
       # print('MA: %2d, ROI:  %.2f%%' % (self.params.period,100*self.roi))
        #self.log('(MA Period%2d) (DIXUpper %2d) ROI:   {:.2f}%' %
         #        (self.params.period, self.params.dixupper, self.roi)
        
        
        
        
        print('(MA Period%2d) (DIXUpper %2d) ROI %.2f%%' %
                 (self.params.period, self.params.dixupper, 100*self.roi,))
        
#runDIXma Strategy Opt
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()    
    
    # Add a strategy
    strats = cerebro.optstrategy(
        DIXmaopt,
        period=range(1,501),
        dixupper=range(41,51)
    ) 
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\DIX MA Testing Backtrader\DIX OHLC.csv')

    # Create a Data Feed
    data = dix_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run(maxcpus=1)
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    # Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


(MA Period 1) (DIXUpper 41) ROI 1.53%
(MA Period 1) (DIXUpper 42) ROI 1.88%
(MA Period 1) (DIXUpper 43) ROI 1.51%
(MA Period 1) (DIXUpper 44) ROI 1.75%
(MA Period 1) (DIXUpper 45) ROI 1.76%
(MA Period 1) (DIXUpper 46) ROI 1.58%
(MA Period 1) (DIXUpper 47) ROI 1.44%
(MA Period 1) (DIXUpper 48) ROI 1.64%
(MA Period 1) (DIXUpper 49) ROI 1.19%
(MA Period 1) (DIXUpper 50) ROI 0.81%
(MA Period 2) (DIXUpper 41) ROI 1.90%
(MA Period 2) (DIXUpper 42) ROI 1.36%
(MA Period 2) (DIXUpper 43) ROI 1.81%
(MA Period 2) (DIXUpper 44) ROI 1.78%
(MA Period 2) (DIXUpper 45) ROI 1.30%
(MA Period 2) (DIXUpper 46) ROI 1.04%
(MA Period 2) (DIXUpper 47) ROI 1.09%
(MA Period 2) (DIXUpper 48) ROI 1.14%
(MA Period 2) (DIXUpper 49) ROI 0.76%
(MA Period 2) (DIXUpper 50) ROI 0.56%
(MA Period 3) (DIXUpper 41) ROI 1.93%
(MA Period 3) (DIXUpper 42) ROI 1.49%
(MA Period 3) (DIXUpper 43) ROI 1.29%
(MA Period 3) (DIXUpper 44) ROI 1.32%
(MA Period 3) (DIXUpper 45) ROI 1.03%
(MA Period 3) (DIXUpper 46) ROI 0.98%
(MA Period 3

(MA Period22) (DIXUpper 47) ROI 0.45%
(MA Period22) (DIXUpper 48) ROI -0.01%
(MA Period22) (DIXUpper 49) ROI 0.00%
(MA Period22) (DIXUpper 50) ROI 0.00%
(MA Period23) (DIXUpper 41) ROI 1.49%
(MA Period23) (DIXUpper 42) ROI 1.57%
(MA Period23) (DIXUpper 43) ROI 1.07%
(MA Period23) (DIXUpper 44) ROI 1.52%
(MA Period23) (DIXUpper 45) ROI 1.16%
(MA Period23) (DIXUpper 46) ROI 0.67%
(MA Period23) (DIXUpper 47) ROI 0.42%
(MA Period23) (DIXUpper 48) ROI 0.04%
(MA Period23) (DIXUpper 49) ROI 0.00%
(MA Period23) (DIXUpper 50) ROI 0.00%
(MA Period24) (DIXUpper 41) ROI 1.46%
(MA Period24) (DIXUpper 42) ROI 1.57%
(MA Period24) (DIXUpper 43) ROI 1.15%
(MA Period24) (DIXUpper 44) ROI 1.56%
(MA Period24) (DIXUpper 45) ROI 0.86%
(MA Period24) (DIXUpper 46) ROI 0.72%
(MA Period24) (DIXUpper 47) ROI 0.36%
(MA Period24) (DIXUpper 48) ROI 0.04%
(MA Period24) (DIXUpper 49) ROI 0.00%
(MA Period24) (DIXUpper 50) ROI 0.00%
(MA Period25) (DIXUpper 41) ROI 1.37%
(MA Period25) (DIXUpper 42) ROI 1.63%
(MA Period2

(MA Period44) (DIXUpper 43) ROI 1.41%
(MA Period44) (DIXUpper 44) ROI 1.15%
(MA Period44) (DIXUpper 45) ROI 0.71%
(MA Period44) (DIXUpper 46) ROI 0.28%
(MA Period44) (DIXUpper 47) ROI 0.20%
(MA Period44) (DIXUpper 48) ROI 0.00%
(MA Period44) (DIXUpper 49) ROI 0.00%
(MA Period44) (DIXUpper 50) ROI 0.00%
(MA Period45) (DIXUpper 41) ROI 1.79%
(MA Period45) (DIXUpper 42) ROI 1.52%
(MA Period45) (DIXUpper 43) ROI 1.47%
(MA Period45) (DIXUpper 44) ROI 1.17%
(MA Period45) (DIXUpper 45) ROI 0.69%
(MA Period45) (DIXUpper 46) ROI 0.27%
(MA Period45) (DIXUpper 47) ROI 0.17%
(MA Period45) (DIXUpper 48) ROI 0.00%
(MA Period45) (DIXUpper 49) ROI 0.00%
(MA Period45) (DIXUpper 50) ROI 0.00%
(MA Period46) (DIXUpper 41) ROI 1.88%
(MA Period46) (DIXUpper 42) ROI 1.59%
(MA Period46) (DIXUpper 43) ROI 1.50%
(MA Period46) (DIXUpper 44) ROI 1.09%
(MA Period46) (DIXUpper 45) ROI 0.77%
(MA Period46) (DIXUpper 46) ROI 0.32%
(MA Period46) (DIXUpper 47) ROI 0.19%
(MA Period46) (DIXUpper 48) ROI 0.00%
(MA Period46

(MA Period65) (DIXUpper 49) ROI 0.00%
(MA Period65) (DIXUpper 50) ROI 0.00%
(MA Period66) (DIXUpper 41) ROI 1.46%
(MA Period66) (DIXUpper 42) ROI 1.37%
(MA Period66) (DIXUpper 43) ROI 1.15%
(MA Period66) (DIXUpper 44) ROI 0.97%
(MA Period66) (DIXUpper 45) ROI 0.14%
(MA Period66) (DIXUpper 46) ROI 0.09%
(MA Period66) (DIXUpper 47) ROI 0.02%
(MA Period66) (DIXUpper 48) ROI 0.00%
(MA Period66) (DIXUpper 49) ROI 0.00%
(MA Period66) (DIXUpper 50) ROI 0.00%
(MA Period67) (DIXUpper 41) ROI 1.58%
(MA Period67) (DIXUpper 42) ROI 1.30%
(MA Period67) (DIXUpper 43) ROI 1.09%
(MA Period67) (DIXUpper 44) ROI 1.00%
(MA Period67) (DIXUpper 45) ROI 0.26%
(MA Period67) (DIXUpper 46) ROI 0.09%
(MA Period67) (DIXUpper 47) ROI 0.02%
(MA Period67) (DIXUpper 48) ROI 0.00%
(MA Period67) (DIXUpper 49) ROI 0.00%
(MA Period67) (DIXUpper 50) ROI 0.00%
(MA Period68) (DIXUpper 41) ROI 1.50%
(MA Period68) (DIXUpper 42) ROI 1.30%
(MA Period68) (DIXUpper 43) ROI 1.06%
(MA Period68) (DIXUpper 44) ROI 0.98%
(MA Period68

(MA Period87) (DIXUpper 45) ROI 0.15%
(MA Period87) (DIXUpper 46) ROI 0.13%
(MA Period87) (DIXUpper 47) ROI 0.00%
(MA Period87) (DIXUpper 48) ROI 0.00%
(MA Period87) (DIXUpper 49) ROI 0.00%
(MA Period87) (DIXUpper 50) ROI 0.00%
(MA Period88) (DIXUpper 41) ROI 1.89%
(MA Period88) (DIXUpper 42) ROI 1.44%
(MA Period88) (DIXUpper 43) ROI 1.44%
(MA Period88) (DIXUpper 44) ROI 0.84%
(MA Period88) (DIXUpper 45) ROI 0.14%
(MA Period88) (DIXUpper 46) ROI 0.13%
(MA Period88) (DIXUpper 47) ROI 0.00%
(MA Period88) (DIXUpper 48) ROI 0.00%
(MA Period88) (DIXUpper 49) ROI 0.00%
(MA Period88) (DIXUpper 50) ROI 0.00%
(MA Period89) (DIXUpper 41) ROI 1.86%
(MA Period89) (DIXUpper 42) ROI 1.45%
(MA Period89) (DIXUpper 43) ROI 1.38%
(MA Period89) (DIXUpper 44) ROI 0.76%
(MA Period89) (DIXUpper 45) ROI 0.17%
(MA Period89) (DIXUpper 46) ROI 0.15%
(MA Period89) (DIXUpper 47) ROI 0.00%
(MA Period89) (DIXUpper 48) ROI 0.00%
(MA Period89) (DIXUpper 49) ROI 0.00%
(MA Period89) (DIXUpper 50) ROI 0.00%
(MA Period90

...

(MA Period404) (DIXUpper 41) ROI 1.36%
(MA Period404) (DIXUpper 42) ROI 1.29%
(MA Period404) (DIXUpper 43) ROI 0.66%
(MA Period404) (DIXUpper 44) ROI 0.09%
(MA Period404) (DIXUpper 45) ROI 0.00%
(MA Period404) (DIXUpper 46) ROI 0.00%
(MA Period404) (DIXUpper 47) ROI 0.00%
(MA Period404) (DIXUpper 48) ROI 0.00%
(MA Period404) (DIXUpper 49) ROI 0.00%
(MA Period404) (DIXUpper 50) ROI 0.00%
(MA Period405) (DIXUpper 41) ROI 1.47%
(MA Period405) (DIXUpper 42) ROI 1.45%
(MA Period405) (DIXUpper 43) ROI 0.66%
(MA Period405) (DIXUpper 44) ROI 0.08%
(MA Period405) (DIXUpper 45) ROI 0.00%
(MA Period405) (DIXUpper 46) ROI 0.00%
(MA Period405) (DIXUpper 47) ROI 0.00%
(MA Period405) (DIXUpper 48) ROI 0.00%
(MA Period405) (DIXUpper 49) ROI 0.00%
(MA Period405) (DIXUpper 50) ROI 0.00%
(MA Period406) (DIXUpper 41) ROI 1.43%
(MA Period406) (DIXUpper 42) ROI 1.43%
(MA Period406) (DIXUpper 43) ROI 0.68%
(MA Period406) (DIXUpper 44) ROI 0.08%
(MA Period406) (DIXUpper 45) ROI 0.00%
(MA Period406) (DIXUpper 

(MA Period425) (DIXUpper 42) ROI 0.83%
(MA Period425) (DIXUpper 43) ROI 0.52%
(MA Period425) (DIXUpper 44) ROI 0.09%
(MA Period425) (DIXUpper 45) ROI 0.00%
(MA Period425) (DIXUpper 46) ROI 0.00%
(MA Period425) (DIXUpper 47) ROI 0.00%
(MA Period425) (DIXUpper 48) ROI 0.00%
(MA Period425) (DIXUpper 49) ROI 0.00%
(MA Period425) (DIXUpper 50) ROI 0.00%
(MA Period426) (DIXUpper 41) ROI 1.22%
(MA Period426) (DIXUpper 42) ROI 0.74%
(MA Period426) (DIXUpper 43) ROI 0.51%
(MA Period426) (DIXUpper 44) ROI 0.08%
(MA Period426) (DIXUpper 45) ROI 0.00%
(MA Period426) (DIXUpper 46) ROI 0.00%
(MA Period426) (DIXUpper 47) ROI 0.00%
(MA Period426) (DIXUpper 48) ROI 0.00%
(MA Period426) (DIXUpper 49) ROI 0.00%
(MA Period426) (DIXUpper 50) ROI 0.00%
(MA Period427) (DIXUpper 41) ROI 1.14%
(MA Period427) (DIXUpper 42) ROI 0.68%
(MA Period427) (DIXUpper 43) ROI 0.51%
(MA Period427) (DIXUpper 44) ROI 0.08%
(MA Period427) (DIXUpper 45) ROI 0.00%
(MA Period427) (DIXUpper 46) ROI 0.00%
(MA Period427) (DIXUpper 

(MA Period446) (DIXUpper 43) ROI 0.59%
(MA Period446) (DIXUpper 44) ROI 0.00%
(MA Period446) (DIXUpper 45) ROI 0.00%
(MA Period446) (DIXUpper 46) ROI 0.00%
(MA Period446) (DIXUpper 47) ROI 0.00%
(MA Period446) (DIXUpper 48) ROI 0.00%
(MA Period446) (DIXUpper 49) ROI 0.00%
(MA Period446) (DIXUpper 50) ROI 0.00%
(MA Period447) (DIXUpper 41) ROI 1.51%
(MA Period447) (DIXUpper 42) ROI 0.87%
(MA Period447) (DIXUpper 43) ROI 0.59%
(MA Period447) (DIXUpper 44) ROI 0.00%
(MA Period447) (DIXUpper 45) ROI 0.00%
(MA Period447) (DIXUpper 46) ROI 0.00%
(MA Period447) (DIXUpper 47) ROI 0.00%
(MA Period447) (DIXUpper 48) ROI 0.00%
(MA Period447) (DIXUpper 49) ROI 0.00%
(MA Period447) (DIXUpper 50) ROI 0.00%
(MA Period448) (DIXUpper 41) ROI 1.51%
(MA Period448) (DIXUpper 42) ROI 0.78%
(MA Period448) (DIXUpper 43) ROI 0.59%
(MA Period448) (DIXUpper 44) ROI 0.00%
(MA Period448) (DIXUpper 45) ROI 0.00%
(MA Period448) (DIXUpper 46) ROI 0.00%
(MA Period448) (DIXUpper 47) ROI 0.00%
(MA Period448) (DIXUpper 

(MA Period467) (DIXUpper 44) ROI 0.00%
(MA Period467) (DIXUpper 45) ROI 0.00%
(MA Period467) (DIXUpper 46) ROI 0.00%
(MA Period467) (DIXUpper 47) ROI 0.00%
(MA Period467) (DIXUpper 48) ROI 0.00%
(MA Period467) (DIXUpper 49) ROI 0.00%
(MA Period467) (DIXUpper 50) ROI 0.00%
(MA Period468) (DIXUpper 41) ROI 1.66%
(MA Period468) (DIXUpper 42) ROI 0.86%
(MA Period468) (DIXUpper 43) ROI 0.74%
(MA Period468) (DIXUpper 44) ROI 0.00%
(MA Period468) (DIXUpper 45) ROI 0.00%
(MA Period468) (DIXUpper 46) ROI 0.00%
(MA Period468) (DIXUpper 47) ROI 0.00%
(MA Period468) (DIXUpper 48) ROI 0.00%
(MA Period468) (DIXUpper 49) ROI 0.00%
(MA Period468) (DIXUpper 50) ROI 0.00%
(MA Period469) (DIXUpper 41) ROI 1.70%
(MA Period469) (DIXUpper 42) ROI 0.83%
(MA Period469) (DIXUpper 43) ROI 0.70%
(MA Period469) (DIXUpper 44) ROI 0.00%
(MA Period469) (DIXUpper 45) ROI 0.00%
(MA Period469) (DIXUpper 46) ROI 0.00%
(MA Period469) (DIXUpper 47) ROI 0.00%
(MA Period469) (DIXUpper 48) ROI 0.00%
(MA Period469) (DIXUpper 

(MA Period488) (DIXUpper 45) ROI 0.00%
(MA Period488) (DIXUpper 46) ROI 0.00%
(MA Period488) (DIXUpper 47) ROI 0.00%
(MA Period488) (DIXUpper 48) ROI 0.00%
(MA Period488) (DIXUpper 49) ROI 0.00%
(MA Period488) (DIXUpper 50) ROI 0.00%
(MA Period489) (DIXUpper 41) ROI 1.63%
(MA Period489) (DIXUpper 42) ROI 0.73%
(MA Period489) (DIXUpper 43) ROI 0.51%
(MA Period489) (DIXUpper 44) ROI 0.00%
(MA Period489) (DIXUpper 45) ROI 0.00%
(MA Period489) (DIXUpper 46) ROI 0.00%
(MA Period489) (DIXUpper 47) ROI 0.00%
(MA Period489) (DIXUpper 48) ROI 0.00%
(MA Period489) (DIXUpper 49) ROI 0.00%
(MA Period489) (DIXUpper 50) ROI 0.00%
(MA Period490) (DIXUpper 41) ROI 1.63%
(MA Period490) (DIXUpper 42) ROI 0.59%
(MA Period490) (DIXUpper 43) ROI 0.40%
(MA Period490) (DIXUpper 44) ROI 0.00%
(MA Period490) (DIXUpper 45) ROI 0.00%
(MA Period490) (DIXUpper 46) ROI 0.00%
(MA Period490) (DIXUpper 47) ROI 0.00%
(MA Period490) (DIXUpper 48) ROI 0.00%
(MA Period490) (DIXUpper 49) ROI 0.00%
(MA Period490) (DIXUpper 